# Pipeline overview:
1. **Find a query protein in the orthoDB database** (retrieve the corresponding orthoDB ID)
   - you can find a protein by looking up its uniprot ID in the orthoDB database. You can also set the orthoDB ID manually if you already know it. 
   - *Note: If it can't find an orthoDB ID for a given UniprotID, it doesn't mean that the protein is absent from the orthoDB. It could still be present but was retrieved from a different database and a uniprot ID was not mapped to it. View OrthoDB documentation for more info on where the sequences come from. I have not solved this problem. In the future, it would be nice to develop a way to search for the actual full length sequence using blast or something, if it fails to find the uniprot ID*
2. **Retrieve the orthoDB-defined groups of homologous sequences** (orthogroup IDs) containing the query protein
3. **Select a group based on phylogenetic level**
4. **Filter** out sequences in the group that are too short (relative to the length of the query sequence) or that contain non amino acid characters
5. **Filter to least divergent orthologs (LDOs)**:
   - For each organism in the group, select the sequence that is most similar to the query sequence such that there is only one sequence per organism
6. **Cluster the filtered LDOs using CD-HIT**
7. **Align the clustered sequences** (uses MAFFT by default)
8. **output** the alignment and the ortholog group information in a directory structure that is compatible with the conservation analysis pipeline (link)

---

You don't have to worry about using any of this code directly, since you should be able to just run the main script (`../../orthodb_tools/scripts/orthogroup_pipeline.py`) but it might be useful to understand what is going on under the hood if you want to write your own script.<br>
The main script in `../../orthodb_tools/scripts/orthogroup_pipeline.py` combines all of the steps in a single script, using a configuration file to specify all of the parameters.<br>
This notebook will walk through all of the steps in the pipeline manually

# imports

In [1]:
from Bio import AlignIO, SeqIO
import orthodb_tools.env_variables.env_variables as env

%load_ext autoreload
%autoreload 2

# Run parameters

In [2]:
min_fraction_of_query_length = 0.5
level='Vertebrata'
LDO_selection_method = 'alfpy_google_distance'

# 1. find a query protein in the orthoDB database

## search for the uniprot id: Q8TC90

In [3]:
from orthodb_tools.orthogroup_processing import uniprotid_search

In [4]:
uniprot_id = 'Q8TC90'
odb_gene_id = uniprotid_search.uniprotid_2_odb_gene_id(uniprot_id)
print()
print(f'Uniprot ID: {uniprot_id} -> ODB gene ID: {odb_gene_id}')

Multiple matches for `Q8TC90` in table
['9606_0:002f40', '9606_0:002f40']
choosing a single id from multiple matches by "longest"
choosing 9606_0:002f40 as the longest sequence

Uniprot ID: Q8TC90 -> ODB gene ID: 9606_0:002f40


# 2. get the ortholog groups (og_ids) that contain the query protein

In [5]:
from orthodb_tools.orthogroup_processing import og_selection
from orthodb_tools import sql_queries

In [6]:
og_ids = sql_queries.odb_gene_id_2_ogid_list(odb_gene_id)
print(og_ids)

['21856at40674', '18706at314295', '18706at9604', '622381at9347', '96736at9443', '96736at314146', '351562at32523', '1771281at33208', '869863at7742', '4642869at2759']


In [7]:
# og_id information
og_selection.get_available_ogs(odb_gene_id)

,OG id,level NCBI tax id,level name,total non-redundant count of species underneath,OG name
2,18706at9604,9604,Hominidae,5,Coiled-coil domain-containing glutamate-rich p...
1,18706at314295,314295,Hominoidea,7,Coiled-coil domain-containing glutamate-rich p...
4,96736at9443,9443,Primates,30,coiled-coil glutamate rich protein 1
5,96736at314146,314146,Euarchontoglires,70,coiled-coil glutamate rich protein 1
3,622381at9347,9347,Eutheria,182,coiled-coil glutamate rich protein 1
0,21856at40674,40674,Mammalia,191,coiled-coil glutamate rich protein 1
6,351562at32523,32523,Tetrapoda,325,coiled-coil glutamate rich protein 1
8,869863at7742,7742,Vertebrata,470,coiled-coil glutamate rich protein 1
7,1771281at33208,33208,Metazoa,817,coiled-coil glutamate rich protein 1
9,4642869at2759,2759,Eukaryota,1952,coiled-coil glutamate rich protein 1


# 3. select a group and retrieve the sequences

## let's select the 'Vertebrata' group

In [8]:
ogid, oglevel = og_selection.select_OG_by_level_name(odb_gene_id=odb_gene_id, level_name='Vertebrata')
print(ogid)
print(oglevel)

869863at7742
Vertebrata


In [9]:
# if we provide a level name that isn't found in the database, we get an error
og_selection.select_OG_by_level_name(odb_gene_id=odb_gene_id, level_name='Bacteria')

ValueError: No OGs found for 9606_0:002f40 with level name `Bacteria`. available levels are: ['Hominidae', 'Hominoidea', 'Primates', 'Euarchontoglires', 'Eutheria', 'Mammalia', 'Tetrapoda', 'Vertebrata', 'Metazoa', 'Eukaryota']

## Get the sequences in the group

In [10]:
group_members = sql_queries.ogid_2_odb_gene_id_list(ogid)
ODB_DATABASE = env.orthoDBDatabase()
sequence_dict = ODB_DATABASE.get_sequences_from_list_of_seq_ids(group_members)
query_seqrecord = sequence_dict[odb_gene_id]
min_length = min_fraction_of_query_length * len(query_seqrecord)

In [11]:
print(query_seqrecord)

ID: 9606_0:002f40
Name: 9606_0:002f40
Description: 9606_0:002f40	9606_0
Number of features: 0
Seq('MTQTLDTREDPLNLGGGGGGGCGCGWAHSASLSSWSSCHRRRPGAPAYNRPHRY...FNC')


# 4. filter out sequences that are too short or contain non amino acid characters

In [12]:
import orthodb_tools.orthogroup_processing.filters as filters

In [15]:
print(len(sequence_dict))
filtered_sequence_dict = filters.filter_seqs_with_nonaa_chars(
    sequence_dict,
)
print(len(filtered_sequence_dict))
filtered_sequence_dict = filters.filter_shorter_sequences(
    filtered_sequence_dict,
    min_length=min_length,
)
if query_seqrecord.id not in filtered_sequence_dict:
    filtered_sequence_dict[query_seqrecord.id] = copy.deepcopy(query_seqrecord)

print(len(filtered_sequence_dict))

349
336
308


# 5. find least divergent orthologs (LDOs)

In [16]:
import orthodb_tools.orthogroup_processing.find_LDOs as find_LDOs

In [17]:
pid_df, ldos = find_LDOs.find_LDOs_main(
    seqrecord_dict=filtered_sequence_dict,
    query_seqrecord=query_seqrecord,
    pid_method = LDO_selection_method,
)

comparing sequences using alignment free comparison (alfpy google distance)


In [18]:
# sequence similarity matrix
pid_df.drop(columns=['sequence'])

,id,organism,PID
0,10020_0:000423,10020_0,0.662710
1,10029_0:0001d8,10029_0,0.651964
2,10036_0:002a79,10036_0,0.661279
3,10041_0:003b55,10041_0,0.659235
4,10047_0:00520a,10047_0,0.644568
...,...,...,...
303,9978_0:000a44,9978_0,0.618854
304,9986_0:000f1c,9986_0,0.671115
305,9994_0:0002f8,9994_0,0.696887
306,9995_0:0035ea,9995_0,0.701641


In [19]:
ldo_seqrecord_dict = ODB_DATABASE.get_sequences_from_list_of_seq_ids(ldos)

In [20]:
print(len(filtered_sequence_dict))
print(len(ldos))

308
295


# 6. cluster the LDOs using CD-HIT

In [21]:
import orthodb_tools.orthogroup_processing.cluster as cluster

In [22]:
cluster_cmd, clustered_ldo_seqrec_dict = cluster.cdhit_main(ldo_seqrecord_dict, odb_gene_id)

Program: CD-HIT, V4.8.1, Apr 11 2024, 12:09:18
Command: cd-hit -i
         /var/folders/q4/k476_qrd3jvdvzwd6lq30kqc0000gn/T/tmpo_ow3f1f
         -o
         /var/folders/q4/k476_qrd3jvdvzwd6lq30kqc0000gn/T/tmpo_ow3f1f-cdhit.fa
         -M 0 -d 0 -g 1

Started: Wed Jul 31 17:19:39 2024
                            Output                              
----------------------------------------------------------------
total seq: 295
longest and shortest : 554 and 203
Total letters: 97034
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 16M = 16M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 81M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 249777000

comparing sequences from          0  to        295

      295  finished        143  clusters

Approximated maximum memory consumption: 81M
writing new database
writing clustering informat

In [23]:
print(len(clustered_ldo_seqrec_dict))

143


# 7. align the clustered sequences

In [24]:
import orthodb_tools.tools.cli_wrappers as cli_wrappers

In [25]:
mafftcmd, aligned_seqs_dict = cli_wrappers.mafft_align_wrapper(
    list(clustered_ldo_seqrec_dict.values()),
    n_align_threads=8,
)

In [26]:
aln = AlignIO.MultipleSeqAlignment(list(aligned_seqs_dict.values()))
print(aln[0:50, 175:220])

Alignment with 50 rows and 45 columns
-----------ASPSPLPPWSPCH------------------RRR 38626_0:002d6b
-----------ASPSSLPPWSPCH------------------RRR 9337_0:0036f4
-----------ASPSPLPPWSPCH------------------RRR 191870_0:004228
-----------ASPSPLPPWSPCH------------------RRR 13616_0:004984
-----------ASPSPLPPWSPCH------------------RRR 29139_0:004c06
-----------ASPSPLPPWSPCH------------------RRR 33562_0:00480c
-----------SSPAAGPPPGACHG-----------------RGR 9258_0:002c95
-----------APPAPLRTWSTCH------------------RRR 286419_0:002935
-----------SSSAPLGTWSSCR------------------RRR 34839_0:00034e
-----------ASSAPLGTWSTCH------------------RRR 36723_0:001c9a
-----------ASSAGLGTWSSCP------------------RRR 143302_0:000941
-----------SSSVPLRTWSSYH------------------RRR 105255_0:002d50
-----------ATSAPLRTSYTCH------------------RRR 9662_0:001c88
-----------APPAPLRTWSTCH------------------RRR 494514_0:004bb3
-----------ASPSALPPWSPCH------------------RRR 9305_0:00441e
-----------ASSAPLGTWSSWH------------------RRR 

# running the whole pipeline

the `orthodb_tools.orthogroup_pipeline` function runs the entire pipeline in one go, using a configuration object (defined in `orthodb_tools.config.orthodb_pipeline_parameters.py`) to specify all of the parameters.

In [27]:
import orthodb_tools
import orthodb_tools.config.orthodb_pipeline_parameters as conf

In [28]:
processing_params = {
    "overwrite": True,
    "filter_params": {
        "min_fraction_shorter_than_query": 0.5,
    },
    "og_select_params": {
        "OG_selection_method": "level_name",
        "OG_level_name": 'Vertebrata',
    },
    "ldo_select_params": {
        "LDO_selection_method": "alfpy_google_distance",
    },
    "align_params": {
        "align": True,
        "n_align_threads": 8,
    },
    "main_output_folder": "output",
    "write_files": True,
}

`orthodb_tools` has a `load_config` function that loads a configuration file into a `conf.PipelineParams` object.<br> Here we go straight from a dictionary so I'll use the `conf.PipelineParams` object directly

In [29]:
uniprot_id = 'Q8TC90'
config = conf.PipelineParams.from_dict(processing_params)

In [30]:
filename, output_dict = orthodb_tools.orthogroup_pipeline(config, uniprot_id=uniprot_id)

Multiple matches for `Q8TC90` in table
['9606_0:002f40', '9606_0:002f40']
choosing a single id from multiple matches by "longest"
choosing 9606_0:002f40 as the longest sequence
comparing sequences using alignment free comparison (alfpy google distance)
Program: CD-HIT, V4.8.1, Apr 11 2024, 12:09:18
Command: cd-hit -i
         /var/folders/q4/k476_qrd3jvdvzwd6lq30kqc0000gn/T/tmp11pg86i8
         -o
         /var/folders/q4/k476_qrd3jvdvzwd6lq30kqc0000gn/T/tmp11pg86i8-cdhit.fa
         -M 0 -d 0 -g 1

Started: Wed Jul 31 17:22:58 2024
                            Output                              
----------------------------------------------------------------
total seq: 295
longest and shortest : 554 and 203
Total letters: 97034
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 16M = 16M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 81M

Table limit with the given memory limit:
Max number of represe

The returned dictionary is essentially the same information that is saved to the json file

In [31]:
for i in output_dict.keys(): print(i)

query_odb_gene_id
query_sequence_str
ogid
oglevel
sequences
sequences_filtered
sequences_ldos
sequences_clustered_ldos
cdhit_command
species_map
query_uniprot_id
processing params
alignment_clustered_ldos_file
alignment_clustered_ldos_command


sequences can be loaded back into memory from the lists of sequence ids (`sequences*` keys)
<br>
for example:

In [32]:
print(output_dict['sequences_clustered_ldos'][0:10])

['38626_0:002d6b', '9337_0:0036f4', '191870_0:004228', '13616_0:004984', '29139_0:004c06', '33562_0:00480c', '9258_0:002c95', '286419_0:002935', '34839_0:00034e', '36723_0:001c9a']


In [33]:
clust_ldo_sequences = ODB_DATABASE.get_sequences_from_list_of_seq_ids(output_dict['sequences_clustered_ldos'])
for i in list(clust_ldo_sequences.values())[0:10]: print(i.seq)

MALALDRRADPLNLGGGWASPSPLPPWSPCHRRRRSTAISRRRHRSGPKSEYAELRKQAKHQHDPGPPWPHAQRRPERSASPTCWGPGGWWTPGPWRCPPVEVWKPPGSVQVVRVYGLRPPCPCCCSCWSGPDKPGPGRLYRKKKKKRWGRKGRGVRRGRPRRPLQNSPPVDLSALLRPVNLYGRRAPGMRAPRNTTQFIMNQVYEDMRKQEKQERRQEALRARQAAAAAAAAATAGQAPGVPAAAAPLDCSGVLQPVRSRDDEDKEMWLLQETFCSFVQNQLYCLVPSPAAAVAEDEEEENEDGACAPPFSAGGGIEEEDGEEEEEEKDEDEEEEEEEEREEDEEEDLEAREEEEEEEDNEEEDEEEEERYGEEEDESEEGDEDEEFKEDENKEELKEEDDSEEEERRRTPTLEAAPTQALSKGLAPASTSEETPAQTLDEVLNSMPPPQTSTLDGTPLQAPDKGIAVASSPLIPSPPPVSETIHQLANVTAYVPPEIQQSLDSLAKLLPRLMADAQNDDDDDDDDDDDDIDKKEINETKESKEKKEEEEGEI
MALALDRRADPLNLGGGWASPSSLPPWSPCHRRRRSTAMSRRRHRSGPKLEYEELRKQAKHQHDPGPPWPHTQRRPERPASLSCWGPGGCWTPGPWRCPPMEVWKPPGRVQVVRVYGLRPPCPCCCSCWSGPDKPGPGRLYRKKKKKRWGRKGRGVRRGHPRRPLQSSPPVDLSALLRPVNLYGRRAPGMRAPRNTTQFIMNQVYEDMRKQEKQERQQEALRARQATAAATSAAAAAAAAAAARQEPGAAAAATPQDGSGALQPVLSRGEEDKEMQLLQGTFYSFVQKQLYCLVPSPAAAVDEDEDEEEDEGQALRLSLQQEGIEEDGEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEDEEKDEDDDEEEEEEEEEKEEEGEEDEEEEEEVKEEEEEEEEEEEEEEEEEDSEEEDEEEERYGEEEDESEEEGDEEEEFKEDET